In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "setup_environment"
   },
   "outputs": [],
   "source": [
    "!pip install web3 eth-account pandas numpy torch tensorflow fastapi uvicorn python-dotenv aiohttp prometheus-client optuna xgboost redis boto3 pytest sqlite3\n",
    "!pip install --upgrade \"jax[cuda12_pip]\" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html\n",
    "import os\n",
    "import yaml\n",
    "from dotenv import load_dotenv\n",
    "import uvicorn\n",
    "from threading import Thread\n",
    "import prometheus_client\n",
    "import warnings\n",
    "import boto3\n",
    "import sqlite3\n",
    "warnings.filterwarnings('ignore')\n",
    "\n",
    "# Load environment variables and settings\n",
    "load_dotenv()\n",
    "with open('settings.yaml', 'r') as f:\n",
    "    settings = yaml.safe_load(f)\n",
    "os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # A100 GPU\n",
    "\n",
    "# Initialize AWS KMS client\n",
    "kms = boto3.client('kms', region_name='us-east-1')\n",
    "encrypted_private_key = os.getenv('ENCRYPTED_PRIVATE_KEY')\n",
    "response = kms.decrypt(CiphertextBlob=bytes.fromhex(encrypted_private_key))\n",
    "os.environ['PRIVATE_KEY'] = response['Plaintext'].decode()\n",
    "\n",
    "# Initialize SQLite database\n",
    "conn = sqlite3.connect('token_cache.db')\n",
    "cursor = conn.cursor()\n",
    "cursor.execute('''\n",
    "    CREATE TABLE IF NOT EXISTS tokens (\n",
    "        address TEXT PRIMARY KEY,\n",
    "        symbol TEXT,\n",
    "        liquidity REAL,\n",
    "        blacklisted BOOLEAN,\n",
    "        last_updated INTEGER\n",
    "    )\n",
    "''')\n",
    "conn.commit()\n",
    "conn.close()\n",
    "\n",
    "# Start Prometheus metrics server\n",
    "prometheus_client.start_http_server(8001)\n",
    "\n",
    "# Start FastAPI server\n",
    "def run_fastapi():\n",
    "    uvicorn.run(\"api_server:app\", host=\"0.0.0.0\", port=8000, log_level=\"info\", workers=2)\n",
    "\n",
    "fastapi_thread = Thread(target=run_fastapi)\n",
    "fastapi_thread.daemon = True\n",
    "fastapi_thread.start()\n",
    "\n",
    "# Run unit tests\n",
    "!pytest tests/\n",
    "\n",
    "# Run backtesting\n",
    "from backtesting import run_backtest\n",
    "run_backtest()\n",
    "\n",
    "# Start pipeline\n",
    "from pipeline import main_pipeline\n",
    "main_pipeline()"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.10.12"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}